In [ ]:
# VERVESTACKS GRID VISUALIZATION - FLOW-SCALED WITH SOLAR/WIND
import xlwings as xw
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from spatial_utils import bus_id_to_commodity

print("=== LOADING DATA ===")

# 1. Load Excel data
try:
    wb = xw.Book('_for network diag.xlsx')
    ws = wb.sheets[0]
    data = ws.used_range.options(pd.DataFrame, header=1, index=False).value
    wb.close()  # Close Excel workbook after reading data
    print(f"✅ Excel data loaded: {len(data)} rows")
except Exception as e:
    print(f"❌ Error loading Excel: {e}")
    exit()

# Close the Excel workbook and app to release resources
try:
    wb.close()
    if xw.apps:
        for app in xw.apps:
            if not app.books:
                app.quit()
except Exception as e:
    print(f"Warning: Could not close Excel objects cleanly: {e}")


# 2. Load CHE bus data
try:
    bus_data = pd.read_csv('1_grids/output/CHE/CHE_clustered_buses.csv')
    print(f"✅ Bus data loaded: {len(bus_data)} buses")
except Exception as e:
    print(f"❌ Error loading grid data: {e}")
    exit()

print("\n=== DETECTING COLUMNS ===")

# 3. Detect columns
def find_col(possible_names, columns):
    for name in possible_names:
        for col in columns:
            if str(col).strip().lower() == name:
                return col
    return None

colnames = [str(c).strip() for c in data.columns]
process_col = find_col(['process', 'proc', 'Process'], colnames)
commodity_col = find_col(['commodity', 'comm', 'Commodity'], colnames)
consumption_col = find_col(['consumption'], colnames)
production_col = find_col(['production'], colnames)

print(f"Process column: {process_col}")
print(f"Commodity column: {commodity_col}")
print(f"Consumption column: {consumption_col}")
print(f"Production column: {production_col}")

if not (process_col and commodity_col and (consumption_col or production_col)):
    print(f"❌ Missing required columns!")
    exit()

# 4. Clean data - use basic cleaning (just process + commodity required)
required_cols = [process_col, commodity_col]
data_clean = data.dropna(subset=required_cols)
print(f"✅ Using basic cleaning: {len(data_clean)} rows")

print("\n=== COMMODITY TO LOCATION MAPPING ===")

# 5. Create commodity to bus coordinate mapping
commodity_to_coords = {}

# Build commodity coordinates lookup using spatial utility
for _, bus_row in bus_data.iterrows():
    bus_id = bus_row['bus_id']
    coords = (bus_row['x'], bus_row['y'])
    
    # Transform bus_id to commodity name
    commodity_name = bus_id_to_commodity(bus_id, add_prefix=True)
    commodity_to_coords[commodity_name] = coords

print(f"Created {len(commodity_to_coords)} commodity->coordinate mappings")

print("\n=== PROCESS POSITIONING ===")

# 6. Position processes based on their commodity connections
process_positions = {}
transmission_lines = []

for _, row in data_clean.iterrows():
    proc = row[process_col]
    comm = row[commodity_col]
    prod_val = row.get(production_col, 0) if production_col else 0
    cons_val = row.get(consumption_col, 0) if consumption_col else 0
    
    # Check if commodity has geographic location
    if comm in commodity_to_coords:
        coords = commodity_to_coords[comm]
        
        if proc.startswith('g_'):
            # Transmission processes: collect coordinates and flow values for line drawing
            line_found = False
            for line_entry in transmission_lines:
                if line_entry['process'] == proc:
                    line_entry['all_coords'].append(coords)
                    line_entry['flow_values'].append(max(abs(prod_val), abs(cons_val)))
                    line_found = True
                    break
            
            if not line_found:
                new_line = {
                    'process': proc,
                    'all_coords': [coords],
                    'flow_values': [max(abs(prod_val), abs(cons_val))]
                }
                transmission_lines.append(new_line)
        
        else:
            # Regular processes: position as nodes (inherit location from commodity)
            process_key = f"{proc}_{comm}"  # Unique key for each process-commodity pair
            process_positions[process_key] = {
                'process': proc,
                'commodity': comm,
                'coords': coords,
                'production': prod_val,
                'consumption': cons_val
            }

print(f"Positioned {len(process_positions)} process instances")
print(f"Found {len(transmission_lines)} transmission lines")

print("\n=== CREATING VISUALIZATION ===")

# 7. Create the visualization
fig, ax = plt.subplots(1, 1, figsize=(16, 12))

# Calculate flow-based line thickness scaling
all_flows = []
for line in transmission_lines:
    all_flows.extend(line['flow_values'])
max_flow = max(all_flows) if all_flows else 1
min_flow = min([f for f in all_flows if f > 0]) if all_flows else 1

# Draw transmission lines (g_ processes) with flow-based thickness
transmission_labels = []  # Store labels for callout positioning
for line in transmission_lines:
    proc = line['process']
    coords_list = line['all_coords']
    flow_values = line['flow_values']
    
    # Calculate average flow for this transmission line
    avg_flow = sum(flow_values) / len(flow_values) if flow_values else 1
    
    # Scale line thickness based on flow (1-8 pixel range)
    if max_flow > min_flow:
        thickness = 1 + 7 * (avg_flow - min_flow) / (max_flow - min_flow)
    else:
        thickness = 2
    thickness = max(1, min(8, thickness))  # Clamp between 1-8
    
    # Draw lines between all coordinate pairs for this transmission process
    for i, coord1 in enumerate(coords_list):
        for j, coord2 in enumerate(coords_list):
            if i < j:  # Avoid duplicate lines and self-connections
                ax.plot([coord1[0], coord2[0]], 
                       [coord1[1], coord2[1]], 
                       color='#888888', linewidth=thickness, alpha=0.8, zorder=1)
                
                # Store label position for callout (midpoint of first line only)
                if len(transmission_labels) < 15:  # Limit labels to avoid clutter
                    mid_x = (coord1[0] + coord2[0]) / 2
                    mid_y = (coord1[1] + coord2[1]) / 2
                    transmission_labels.append({
                        'text': proc.split('-')[0],
                        'line_pos': (mid_x, mid_y),
                        'flow': avg_flow
                    })

# Categorize and draw process nodes with flow-based sizing
ep_processes = []
solar_wind_processes = []  # Renamed from distr_processes
demand_processes = []
other_processes = []

# Calculate min/max flows for node scaling
all_node_flows = []
for proc_key, proc_info in process_positions.items():
    flow = max(abs(proc_info['production']), abs(proc_info['consumption']))
    if flow > 0:
        all_node_flows.append(flow)

max_node_flow = max(all_node_flows) if all_node_flows else 1
min_node_flow = min(all_node_flows) if all_node_flows else 1

for proc_key, proc_info in process_positions.items():
    proc = proc_info['process']
    coords = proc_info['coords']
    flow = max(abs(proc_info['production']), abs(proc_info['consumption']))
    
    # Calculate node size based on flow (50-400 pixel range)
    if max_node_flow > min_node_flow and flow > 0:
        size = 50 + 350 * (flow - min_node_flow) / (max_node_flow - min_node_flow)
    else:
        size = 100
    size = max(50, min(400, size))  # Clamp between 50-400
    
    if proc.startswith('ep_'):
        ep_processes.append((coords, proc, size))
    elif proc.startswith('distr'):
        # Determine if it's solar or wind based on process name
        if 'solar' in proc.lower() or 'pv' in proc.lower():
            label = proc.replace('distr', 'solar')
        elif 'wind' in proc.lower():
            label = proc.replace('distr', 'wind')
        else:
            label = proc.replace('distr', 'solar/wind')
        solar_wind_processes.append((coords, label, size))
    elif proc.startswith('e_demand'):
        demand_processes.append((coords, proc, size))
    else:
        other_processes.append((coords, proc, size))

# Draw different process types with callout labels and flow-based sizing
def draw_processes_with_callouts(process_list, color, label_prefix):
    if process_list:
        # Extract coordinates, labels, and sizes
        coords_list = [p[0] for p in process_list]
        labels_list = [p[1] for p in process_list]
        sizes_list = [p[2] for p in process_list]
        
        x_coords = [c[0] for c in coords_list]
        y_coords = [c[1] for c in coords_list]
        
        # Draw nodes with individual sizes
        ax.scatter(x_coords, y_coords, c=color, s=sizes_list, alpha=0.9, 
                  marker='s', edgecolors='white', linewidths=2, zorder=3)
        
        # Create callout labels to avoid overlap
        for i, ((x, y), label, size) in enumerate(process_list):
            # Calculate callout position (offset from node, scaled by node size)
            angle = (i * 45) % 360  # Distribute angles
            offset_dist = 0.15 + (size / 2000)  # Scale offset with node size
            label_x = x + offset_dist * np.cos(np.radians(angle))
            label_y = y + offset_dist * np.sin(np.radians(angle))
            
            # Draw thin line from node to label
            ax.plot([x, label_x], [y, label_y], 
                   color='gray', linewidth=0.5, alpha=0.7, zorder=2)
            
            # Add label at offset position
            ax.text(label_x, label_y, label[:12], fontsize=8, ha='center', va='center',
                   bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.9, edgecolor='gray'))

# Draw process types with flow-based sizing and callouts
draw_processes_with_callouts(ep_processes, '#2E8B57', 'ep_')  # Green generators
draw_processes_with_callouts(solar_wind_processes, '#FFB347', 'solar/wind')  # Orange solar/wind
draw_processes_with_callouts(demand_processes, '#DC143C', 'e_demand')  # Red demand
draw_processes_with_callouts(other_processes, '#2E5BBA', 'other')  # Default blue

# Add transmission line callouts
for i, label_info in enumerate(transmission_labels):
    line_x, line_y = label_info['line_pos']
    text = label_info['text']
    flow = label_info['flow']
    
    # Position callout away from line
    angle = (i * 30) % 360
    offset_dist = 0.2
    callout_x = line_x + offset_dist * np.cos(np.radians(angle))
    callout_y = line_y + offset_dist * np.sin(np.radians(angle))
    
    # Draw thin line from transmission line to label
    ax.plot([line_x, callout_x], [line_y, callout_y], 
           color='gray', linewidth=0.5, alpha=0.7, zorder=2)
    
    # Add transmission label with flow info
    ax.text(callout_x, callout_y, f"{text}\n({flow:.0f})", fontsize=7, 
           ha='center', va='center',
           bbox=dict(boxstyle='round,pad=0.2', facecolor='lightyellow', alpha=0.9, edgecolor='gray'))

print(f"Process breakdown:")
print(f"  - Generators (ep_): {len(ep_processes)}")
print(f"  - Solar/Wind (distr): {len(solar_wind_processes)}")
print(f"  - Demand (e_demand): {len(demand_processes)}")
print(f"  - Transmission lines (g_): {len(transmission_lines)}")
print(f"  - Other processes: {len(other_processes)}")

# Styling
ax.set_title('VerveStacks Grid Architecture - Flow-Scaled\nProcesses at Geographic Locations, Transmission as Lines', 
            fontsize=16, fontweight='bold', pad=20)

# Create legend with flow scaling info
legend_elements = [
    Rectangle((0,0), 1, 1, facecolor='#2E8B57', edgecolor='white', label='Generators (ep_)'),
    Rectangle((0,0), 1, 1, facecolor='#FFB347', edgecolor='white', label='Solar/Wind Collection'),
    Rectangle((0,0), 1, 1, facecolor='#DC143C', edgecolor='white', label='Demand (e_demand)'),
    plt.Line2D([0], [0], color='#888888', linewidth=3, label='Transmission Lines (g_)')
]
ax.legend(handles=legend_elements, loc='upper right', fontsize=11, 
          title='VerveStacks Components\n(Node/Line size ∝ Flow)', title_fontsize=12)

ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ VerveStacks visualization complete!")
print(f"   - Commodities: Invisible (location carriers only)")
print(f"   - Processes: {len(process_positions)} positioned instances")
print(f"   - Transmission: {len(transmission_lines)} lines between coordinates")
